# Fitting to empirical data

This notebook is used to generate maximum likelihood estimates for the parameters of a dynamic model judging from empirically collected data. It's used to produce Table 1, Figure 4, and supplemental contour figures in the accompanying paper.

## Google Colab initialization

This section will help you interface with Google Drive and clone the git repository where the code lives. These steps **aren't necessary if you are running locally**. First, make sure you have opened the notebook in Google Colab (use the below button if ncessary) and logged into your Google account.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/tanderson11/covidhouseholds/blob/main/notebooks/EmpiricalFits.ipynb)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%mkdir /content/gdrive/My\ Drive/github/
%cd /content/gdrive/My\ Drive/github/
# Thayer has his files located here instead
#%cd /content/gdrive/My\ Drive/github/paper_push


In [ ]:
# If you've forked the repository, point to your own username and repository name (if different)
repo_owner="tanderson11"
repository="covidhouseholds"

!git config --global user.email "tanderson11@gmail.com"
!git config --global user.name "Thayer Anderson"

In [ ]:
!git clone https://github.com/tanderson11/covidhouseholds.git

In [ ]:
%cd covidhouseholds/
!ls -a

# >>> TOKEN SETUP: <<<
# this will put your token in the right folder; comment this line out after use to avoid an error message
#!mv ../git_token.py ./
#!cp ../../covidhouseholds/git_token.py ./

#from git_token import git_token

In [ ]:
!git checkout main
!git pull

In [ ]:
%cd ./notebooks

## Module initialization

In [1]:
import numpy as np
import os
import pandas as pd
import pyarrow.parquet as pq
%cd ../
import src.likelihood as likelihood
import src.recipes as recipes

/Users/thayer/develop/covid_households


ModuleNotFoundError: No module named 'likelihood'

## Loading empirical and simulated datasets

In [ ]:
root = os.path.abspath('')
results = recipes.Results.load(os.path.join(root, 'new_parameters/gillespie-s80-p80-SAR/beta_corrections'))
results.find_frequencies(inplace=True)

keys = results.metadata.parameters
keys

In [ ]:
# load the data as a pandas dataframe
population_name = 'Ontario'
if population_name == 'Ontario':
    empirical_df = os.path.join(root, "empirical/Ontario/empirical_df.parquet")
    empirical_df = pq.read_table(empirical_df).to_pandas()
elif population_name == 'Bnei Brak':
    results = recipes.Results.load(os.path.join(root, 'new_parameters/gillespie-s80-p80-SAR/beta_corrections/high_sizes'))
    results.find_frequencies(inplace=True)

    dfs = []
    n_random_dfs = 20
    for i in range(n_random_dfs):
        randomized_df_path = os.path.join(root, f"empirical/BneiBrak/randomized_corrections/randomized_df{i}.parquet")
        randomized_df = pq.read_table(randomized_df_path).to_pandas()
        dfs.append(randomized_df)
    empirical_df = pd.concat(dfs)
    print(empirical_df)
    true_path = os.path.join(root, f"empirical/BneiBrak/empirical_df.parquet")
    bnei_brak_true_hhs = pq.read_table(true_path).to_pandas()
elif population_name == 'Geneva':
    empirical_df = os.path.join(root, "empirical/geneva/empirical_df.parquet")
    empirical_df = pq.read_table(empirical_df).to_pandas()
else:
    assert ValueError("population name is not recognized.")

# the infections column should be integer values
empirical_df['infections'] = empirical_df['infections'].round(0)
empirical_df = empirical_df.astype({'infections': 'int32', 'size': 'int32'})

# for the different parameter values, we don't know their true value, so let's give them the dummy value of 0.
for key in keys:
    empirical_df[key] = 0.

# turn the list of households into counts of the number of infections in different households
empirical_counts = likelihood.counts_from_empirical(empirical_df, keys, sample_only_keys=[])

if population_name == 'Bnei Brak':
    empirical_counts = empirical_counts[empirical_counts.index.get_level_values('size') <= 15]
    empirical_counts = empirical_counts / 20
empirical_counts.tail(50)

In [ ]:
def total_secondary_contacts(empirical_counts):
    counts = empirical_counts.reset_index()
    return ((counts['size'] - 1) * counts['count']).sum()

def secondary_contacts_infected(empirical_counts):
    counts = empirical_counts.reset_index()
    return ((counts['infections'] - 1) * counts['count']).sum()

n = total_secondary_contacts(empirical_counts)
X = secondary_contacts_infected(empirical_counts)

n,X

In [ ]:
def sep(n, X):
    p_hat = X/n
    q_hat = 1 - p_hat
    sep = np.sqrt(p_hat * q_hat/n)

    return sep

# Careful not to use the randomized Bnei Brak data. Here are the actual results:
# Bnei Brak: secondary contacts infected = 975
# Bnei Brak: total # secondary contacts = 2716

sep(n,X) * 2 * 100

In [ ]:
empirical_counts

Sometimes for fun, we want to look at the best fit under a "null hypothesis" of no heterogeneity. In order to do that, we need to select the results of our simulations that have no heterogeneity ($s_{80} = p_{80} = 80\%$)

In [ ]:
freq = results.df['frequency']
s80_l = freq.index.get_level_values(0)
p80_l = freq.index.get_level_values(1)

null_freqs = freq[(s80_l == 0.8) & (p80_l == 0.8)]
null_freqs

## Constructing likelihood surface and finding MLEs

By default, the log likelihood is held in a `Series` of log likelihood values organized around a 4-level index. The first index represent which empirical trial corresponds to that section of data. The next three parameters correspond to the parameter values at which we are estimating the likelihood (ie the model parameters).

In [ ]:
# calculate the log likelihood using the table of frequencies calculated over many households and the empirical counts from the dataset
logl = likelihood.logl_from_frequencies_and_counts(results.df['frequency'], empirical_counts, keys)
# by default indexed by a 'trial' index, but there is only one trial (the 0th) because this is a single dataset
logl = logl.loc[0]
logl

In [ ]:
# "profile" likelihood: looking at fixing two parameters at the MLE and trying to get 95% confidence in the last one
in_95 = likelihood.normalize_probability(logl).sort_values(ascending=False).head()
in_95

The parameters of the maximum likelihood estimate are easy to obtain by just finding the maximum value in the table of likelihood values.

In [ ]:
mle = logl.idxmax()
keys, mle

In [ ]:
null_logl = likelihood.logl_from_frequencies_and_counts(null_freqs, empirical_counts, keys).loc[0]
null_mle = null_logl.idxmax()
null_mle

In [ ]:
confidence_95_mask = likelihood.confidence_mask_from_logl(logl, percentiles=(0.95,)).astype('bool')
confidence_95_intervals = [
    likelihood.confidence_interval_from_confidence_mask(confidence_95_mask, 's80'),
    likelihood.confidence_interval_from_confidence_mask(confidence_95_mask, 'p80'),
    likelihood.confidence_interval_from_confidence_mask(confidence_95_mask, 'SAR'),
]
keys, confidence_95_intervals

## Bar graphs of empirically observed infections and best fit simulated infections

To construct these graphs, we have to organize the empirical data and the simulated data into a single DataFrame and then plot the observed/expected infections.

In [ ]:
# the frequencies at the mle give the proportions of different # of infections in the best fit from the model
best_fit_infections = results.df.loc[mle]['frequency']
best_fit_infections.name = "model"
best_fit_infections

In [ ]:
# the frequencies at the null hypothesis mle give the proportions of different # of infections in the best fit from the model
null_fit_infections = results.df.loc[null_mle]['frequency']
null_fit_infections.name = "null model"

In [ ]:
# the empirical data is observed counts, we convert that to frequencies by dividing the counts by the total number of households of that size
empirical_frequencies = empirical_counts/(empirical_counts.groupby(['sample ' + k for k in keys]+["size"]).sum())
empirical_frequencies.name = 'data'
# we have to index past the dummy key values that we assigned earlier for each parameter in the empirical dataset
empirical_frequencies = empirical_frequencies.loc[0.0, 0.0, 0.0]
empirical_frequencies

In [ ]:
import matplotlib.pyplot as plt
include_null = False
if include_null:
    combined_best_and_empirical = pd.concat([empirical_frequencies, best_fit_infections, null_fit_infections], axis=1)
else:
    combined_best_and_empirical = pd.concat([empirical_frequencies, best_fit_infections], axis=1)
empirical_sizes = empirical_df.groupby('size').size()
n_sizes = len(empirical_sizes.index.unique())

which_sizes = empirical_sizes.index.unique()
which_sizes = [2, 5, 8]

vertical = True

n_cols = 3
n_rows = int(np.ceil(len(which_sizes) / n_cols))

if vertical:
    n_cols, n_rows = n_rows, n_cols


fixed_ylims = False
ylims_by_household_size = {
    2: 6000,
    5: 3000,
    8: 300
}

fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols)
axis_generator = (ax for ax in np.ravel(axes))
cmap = plt.get_cmap("tab10")
color_dict = {'model': cmap(0), 'data': cmap(1), 'null model':cmap(2)}
for k,g in combined_best_and_empirical.groupby('size'):
    if k not in which_sizes:
        continue
    try:
        ax = next(axis_generator)
    except StopIteration:
        break
    try:
        title = f"Household size = {k} (# hh in data = {empirical_sizes[k]})"
    except KeyError:
        continue
    g = g.set_index(g.index.droplevel('size')) * empirical_sizes[k]
    g = g / g.sum()
    g.plot.bar(
        ax=ax,
        figsize=(5*n_cols,n_rows*4),
        title=title,
        xlabel="infections",
        ylabel="fraction",
        sharex=True,
        color=color_dict,
    ),
    if fixed_ylims:
        ax.set_ylim((0,  ylims_by_household_size[k]))
    #plt.ylim((0,6000))

plt.savefig(f'./figures/{population_name}_data.pdf', dpi=400, bbox_inches='tight')

In [ ]:
ecounts = empirical_counts.loc[0.0, 0.0, 0.0]
(ecounts.groupby('size').sum()/(ecounts.groupby('size').sum().sum())).plot.bar(color='green')
plt.ylim(0, 0.5)
plt.savefig(f'./figures/{population_name}_household_histogram.pdf', dpi=400, bbox_inches='tight')

## Contours of the likelihood surface for a given parameter

In [ ]:
prob_df = likelihood.normalize_probability(logl)
prob_df = prob_df.groupby(['s80', 'SAR']).sum()
prob_df = prob_df / prob_df.sum()

In [ ]:
Z = prob_df.unstack()

X,Y = np.meshgrid(Z.columns, Z.index)
#print(self.kwargs)
contourf = plt.contourf(X, Y, Z)